In [1]:
!pip install redis-server redis

import redis_server
!$redis_server.REDIS_SERVER_PATH --daemonize yes

import redis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.5/238.5 KB 17.7 MB/s eta 0:00:00


In [ ]:
# Connect to the Redis server
r = redis.StrictRedis(host='localhost', port=6379, db=0)

# Publisher
def publish_message(channel, message):
    r.publish(channel, message)

# Subscriber
def subscribe_to_channel(channel):
    pubsub = r.pubsub()
    pubsub.subscribe(channel)

    for message in pubsub.listen():
        print(f"Received: {message['data']}")

# Usage
if __name__ == "__main__":
    import threading

    channel = 'my_channel'

    subscriber_thread = threading.Thread(target=subscribe_to_channel, args=(channel,))
    subscriber_thread.start()

    while True:
        msg = input("Enter message: ")
        publish_message(channel, msg)


In [9]:
# Connect to the Redis server
r = redis.StrictRedis(host='localhost', port=6379, db=0)

# Set key-value pairs
r.set('name', 'John Doe')
r.set('age', 30)

# Get values
name = r.get('name')
age = r.get('age')

print(f"Name: {name}, Age: {age}")


Name: b'John Doe', Age: b'30'


In [14]:
# Connect to the Redis server
r = redis.StrictRedis(host='localhost', port=6379, db=0)

# Cache function results
def cache_result(func, key, expire=None):
    result = r.get(key)
    if not result:
        result = func()
        r.set(key, result)
        if expire:
            r.expire(key, expire)
    return result

# Expensive function
def expensive_function():
    # Simulating a time-consuming operation
    import time
    time.sleep(5)
    print("here")
    return "Expensive result"

# Usage
if __name__ == "__main__":
    key = "expensive_function_result"
    result = cache_result(expensive_function, key, expire=10)
    result = cache_result(expensive_function, key, expire=10)
    result = cache_result(expensive_function, key, expire=10)


here


In [2]:
#strings
r = redis.StrictRedis(host='localhost', port=6379, db=0)

# Set and get a string value
r.set('greeting', 'Hello, world!')
greeting = r.get('greeting')
print("Greeting:", greeting.decode())

# Increment a counter
r.incr('counter')
r.incrby('counter', 10)
counter = r.get('counter')
print("Counter:", counter.decode())


Greeting: Hello, world!
Counter: 11


In [3]:
#lists
r = redis.StrictRedis(host='localhost', port=6379, db=0)

# Push elements to a list
r.lpush('fruits', 'apple', 'banana', 'orange')

# Pop elements from a list
fruit = r.rpop('fruits')
print("Fruit:", fruit.decode())

# Get elements from a list
fruits = r.lrange('fruits', 0, -1)
print("Fruits:", [f.decode() for f in fruits])


Fruit: apple
Fruits: ['orange', 'banana']


In [4]:
#sets

r = redis.StrictRedis(host='localhost', port=6379, db=0)

# Add elements to a set
r.sadd('users', 'Alice', 'Bob', 'Charlie', 'Alice')

# Remove an element from a set
r.srem('users', 'Bob')

# Check if an element is a member of a set
is_member = r.sismember('users', 'Charlie')
print("Charlie is a member of users:", is_member)

# Get all elements of a set
users = r.smembers('users')
print("Users:", [user.decode() for user in users])


Charlie is a member of users: True
Users: ['Alice', 'Charlie']


In [5]:
#hashes

r = redis.StrictRedis(host='localhost', port=6379, db=0)

# Set fields in a hash
r.hmset('user:1', {'name': 'Alice', 'age': 30, 'email': 'alice@example.com'})

# Get fields from a hash
user = r.hgetall('user:1')
print("User:", {k.decode(): v.decode() for k, v in user.items()})

# Update a field in a hash
r.hincrby('user:1', 'age', 1)


User: {'name': 'Alice', 'age': '30', 'email': 'alice@example.com'}


<ipython-input-5-f2aaf1256244>:6: DeprecationWarning: Redis.hmset() is deprecated. Use Redis.hset() instead.
  r.hmset('user:1', {'name': 'Alice', 'age': 30, 'email': 'alice@example.com'})


31

In [6]:
#geospatial data

r = redis.StrictRedis(host='localhost', port=6379, db=0)

# Add geospatial data
coords = [(13.361389, 38.115556, "Palermo"),(2.3522, 48.8566, 'Paris'),(12.4964, 41.9028, 'Rome'),(-0.1278, 51.5074, 'London')]
r.geoadd("cities", coords[0])
r.geoadd('cities', coords[1])
r.geoadd('cities', coords[2])
r.geoadd('cities', coords[3])

# Calculate the distance between two cities
distance = r.geodist('cities', 'Paris', 'Rome', unit='km')
print("Distance between Paris and Rome:", distance, "km")

# Find nearby cities
nearby_cities = r.georadius('cities', 2.3522, 48.8566, 1000, unit='km', withdist=True)
print("Cities within 1000 km of Paris:", nearby_cities)


Distance between Paris and Rome: 1105.592 km
Cities within 1000 km of Paris: [[b'Paris', 0.0002], [b'London', 343.653]]


In [7]:
#sorted sets

r = redis.StrictRedis(host='localhost', port=6379, db=0)

# Add members to the sorted set
r.zadd('leaderboard', {'Alice': 1500, 'Bob': 1000, 'Charlie': 1200})

# Increment a member's score
r.zincrby('leaderboard', 500, 'Bob')

# Get the top N members
top_members = r.zrevrange('leaderboard', 0, 2, withscores=True)
print("Top members:", top_members)

# Get a member's rank
rank = r.zrevrank('leaderboard', 'Charlie')
print("Charlie's rank:", rank + 1)

Top members: [(b'Bob', 1500.0), (b'Alice', 1500.0), (b'Charlie', 1200.0)]
Charlie's rank: 3


In [8]:
#bitmaps
r = redis.StrictRedis(host='localhost', port=6379, db=0)

# Set bits for user activity
r.setbit('user1_activity', 1, 1)
r.setbit('user1_activity', 7, 1)

# Check if a user was active on a specific day
active = r.getbit('user1_activity', 1)
print("User 1 active on day 1:", bool(active))

# Count active days for a user
active_days = r.bitcount('user1_activity')
print("User 1 active days:", active_days)


User 1 active on day 1: True
User 1 active days: 2
